<a href="https://colab.research.google.com/github/cuong3004/Trash_Cuong/blob/main/C%C3%A1c_b%C6%B0%E1%BB%9Bc_th%E1%BB%B1c_hi%E1%BB%87n_suy_lu%E1%BA%ADn_yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load model yolov5

```java
Interpreter.Options options=new Interpreter.Options();
GpuDelegate gpuDelegate=new GpuDelegate();
options.addDelegate(gpuDelegate);
options.setNumThreads(4); // set it according to your phone
Interpreter interpreter = new Interpreter(
        loadModelFile(assetManager, "yolov5s-int8.tflite"),
        options
);
// load labelmap
List<String> labelList=loadLabelList(assetManager,"label.txt"); // cavity, normal
```

## Load hàm NMS

```java
import com.example.imagepro.ml.Nms2;
Nms2 nms =  Nms2.newInstance(this);
```

## Xử lý hình ảnh từ camera

```java
// Hình ảnh từ camera được load bằng opencv sẽ trả về đối tượng class Mat
Mat mat_image_org = ...;

// Tính toán vùng đẻ crop một phần hình ảnh
int width=mat_image_org.cols();
int height=mat_image_org.rows();
int minSize = Math.min(width, height);
int cropX = width / 2 - minSize / 2;
int cropY = height / 2 - minSize / 2;

Rect rectCrop = new Rect(cropX, cropY, minSize, minSize);
Mat mat_image = new Mat(mat_image_org,rectCrop); // crop ảnh

// sử lý trong trường hợp bị ngược
Mat rotated_mat_image=new Mat();
Core.flip(mat_image.t(),rotated_mat_image,1); 

// Chuyển đổi Mat sang Bitmap(để đưa vào model)
Bitmap bitmap=null;
bitmap=Bitmap.createBitmap(rotated_mat_image.cols(),rotated_mat_image.rows(),Bitmap.Config.ARGB_8888);
Utils.matToBitmap(rotated_mat_image,bitmap);
```



## Quá trình suy luận

```java
// Các thao tác tiền sử lý như thay đổi kích thước, chuẩn hóa, quantize
ImageProcessor imageProcessor =
    new ImageProcessor.Builder()
        .add(new ResizeOp(192, 192, ResizeOp.ResizeMethod.BILINEAR))
        .add(new NormalizeOp(0f, 255f))
        .add(new QuantizeOp(0,  0.003921568859368563f))
        .add(new CastOp(DataType.UINT8))
        .build();
// Các thao tác hậu sử lý như dequantize
TensorProcessor probabilityProcessor =
    new TensorProcessor.Builder()
        .add(new DequantizeOp(0, 0.007786227390170097f)).build();

TensorImage tensorImage = new TensorImage(DataType.UINT8);
tensorImage.load(scaledBitmap);
tensorImage = imageProcessor.process(tensorImage); // tiền sử lý

// Định nghĩa hình dạng đầu vào và đầu ra của mô hình
TensorBuffer inputFeature = 
    TensorBuffer.createFixedSize(new int[]{1, 192, 192, 3}, DataType.UINT8);
TensorBuffer probabilityBuffer =
    TensorBuffer.createFixedSize(new int[]{1, 2268, 7}, DataType.UINT8);
TensorBuffer inputNMS = 
    TensorBuffer.createFixedSize(new int[]{1, 2268, 7}, DataType.FLOAT32); // NMS

inputFeature.loadBuffer(tensorImage.getBuffer());

// Suy luận
interpreter.run(inputFeature.getBuffer(), probabilityBuffer.getBuffer());
probabilityBuffer = probabilityProcessor.process(probabilityBuffer); // hậu sử lý
// Hậu sử lý NMS
inputNMS.loadBuffer(outputModel);
Nms2.Outputs outputNMS = nms.process(inputNMS);
TensorBuffer outputFeature0 = outputNMS.getOutputFeature0AsTensorBuffer();
TensorBuffer outputFeature1 = outputNMS.getOutputFeature1AsTensorBuffer();
TensorBuffer outputFeature2 = outputNMS.getOutputFeature2AsTensorBuffer();

float[] classes = outputFeature0.getFloatArray(); // class
float[] bboxes = outputFeature1.getFloatArray(); // value_box
float[] scores = outputFeature2.getFloatArray(); // score
```

## Vẽ lên ảnh

```java
for (int i=0;i<classes.length;i++){
    float class_value = classes[i];
    float score_value = scores[i];

    if(score_value>0.5){
        float left= bboxes[i*4+0]*minSize;
        float top= bboxes[i*4+1]*minSize;
        float right= bboxes[i*4+2]*minSize;
        float bottom= bboxes[i*4+3]*minSize;

    if ((int) class_value == 1){ // normal
        Imgproc.rectangle( // vẽ các hộp chữ nhật
            rotated_mat_image,
            new Point(left,top),
            new Point(right,bottom),
            new Scalar(0, 255, 0, 255),2);

        Imgproc.putText( // vẽ chữ
            rotated_mat_image,labelList.get((int) class_value),
            new Point(left,top),3,0.5,
            new Scalar(0, 255, 0, 255),2);
    } else { // Caries
        Imgproc.rectangle(  // vẽ các hộp chữ nhật
            rotated_mat_image,
            new Point(left,top),
            new Point(right,bottom),
            new Scalar(255, 0, 0, 255),2);

        Imgproc.putText( // vẽ chữ
            rotated_mat_image,
            labelList.get((int) class_value),
            new Point(left,top),3,0.5,
            new Scalar(255, 0, 0, 255),2);
    }

Core.flip(rotated_mat_image.t(),mat_image,0);
return mat_image_org;
```